# Workshop Session 4: Neural Networks for Time Series Forecasting

## Welcome to Session 4!

In previous sessions, we explored **statistical models** (Session 1) and **machine learning models** (Sessions 2 & 3). Now we'll dive into **neural networks**!

### What You'll Learn:
1. **NeuralForecast Library** - deep learning made easy for time series
2. **Popular Neural Models** - NBEATS, NHITS
3. **Automatic Hyperparameter Tuning** - using AutoNBEATS and AutoNHITS
4. **Model Comparison** - evaluating neural network performance

### Why Neural Networks?

**Advantages**:
- ✅ Capture **complex non-linear patterns**
- ✅ Learn **temporal dependencies** automatically
- ✅ Handle **multiple time series** efficiently (global models)
- ✅ **Automatic feature learning** (no manual feature engineering)

**Trade-offs**:
- ⚠️ Require more data and computational resources
- ⚠️ Longer training times
- ⚠️ Less interpretable than simpler models

Let's get started!

## 1. Setup and Imports

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# NeuralForecast
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.auto import AutoNBEATS, AutoNHITS

# Metrics
from sklearn.metrics import mean_squared_error

# Plotting
plt.style.use("ggplot")
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

print("All libraries imported successfully!")

2025-10-15 22:42:51,157	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-10-15 22:42:51,436	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


All libraries imported successfully!


## 2. Load Data and Train/Test Split

We'll use the same M5 food sales dataset from previous sessions.

In [2]:
# Load the data
data_path = "/home/filtheo/Cloud-for-AI/workshop_ml_2025/data/converted_df.csv"
df = pd.read_csv(data_path, parse_dates=["date"])

print(f"Dataset shape: {df.shape}")
print(f"Number of stores: {df['unique_id'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\nFirst few rows:")
df.head(10)

Dataset shape: (12100, 3)
Number of stores: 10
Date range: 2013-01-01 00:00:00 to 2016-04-24 00:00:00

First few rows:


,unique_id,date,y
0,CA_1,2013-01-01,1888
1,CA_2,2013-01-01,1320
2,CA_3,2013-01-01,2454
3,CA_4,2013-01-01,1031
4,TX_1,2013-01-01,1607
5,TX_2,2013-01-01,2469
6,TX_3,2013-01-01,1773
7,WI_1,2013-01-01,1365
8,WI_2,2013-01-01,1506
9,WI_3,2013-01-01,1415


In [3]:
# Train/Test Split: last 14 days for testing (same as previous sessions)
test_size = 14
max_date = df["date"].max()
split_date = max_date - pd.Timedelta(days=test_size - 1)

df_train = df[df["date"] < split_date].copy()
df_test = df[df["date"] >= split_date].copy()

print(f"Train/Test Split:")
print("=" * 60)
print(f"Training set: {len(df_train):,} observations")
print(f"Test set: {len(df_test):,} observations")
print(f"\nDate ranges:")
print(f"  Train: {df_train['date'].min()} to {df_train['date'].max()}")
print(f"  Test:  {df_test['date'].min()} to {df_test['date'].max()}")
print(f"\nForecast horizon: {test_size} days")

# Store list
stores = df["unique_id"].unique()
print(f"\nStores: {list(stores)}")

Train/Test Split:
Training set: 11,960 observations
Test set: 140 observations

Date ranges:
  Train: 2013-01-01 00:00:00 to 2016-04-10 00:00:00
  Test:  2016-04-11 00:00:00 to 2016-04-24 00:00:00

Forecast horizon: 14 days

Stores: ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']


## 3. Understanding NeuralForecast Data Format

**NeuralForecast requires:**
- `unique_id`: identifier for each time series
- `ds`: timestamp column (datestamp)
- `y`: target variable

Our data already has `unique_id` and `y`, we just need to rename `date` to `ds`!

In [4]:
# Prepare data for NeuralForecast
df_train_nf = df_train.rename(columns={"date": "ds"})
df_test_nf = df_test.rename(columns={"date": "ds"})

print("Data prepared for NeuralForecast:")
print("=" * 60)
print(df_train_nf.head())
print(f"\nColumns: {list(df_train_nf.columns)}")
print("✅ Format ready: unique_id, ds, y")

Data prepared for NeuralForecast:
  unique_id         ds     y
0      CA_1 2013-01-01  1888
1      CA_2 2013-01-01  1320
2      CA_3 2013-01-01  2454
3      CA_4 2013-01-01  1031
4      TX_1 2013-01-01  1607

Columns: ['unique_id', 'ds', 'y']
✅ Format ready: unique_id, ds, y


## 4. Introduction to Neural Network Models

We'll explore two popular neural network architectures:

### 1. **NBEATS** (Neural Basis Expansion Analysis)
- **Key Idea**: Decomposes forecasts into trend and seasonality components using basis functions
- **Interpretable**: Can visualize trend/seasonal contributions
- **Architecture**: Deep residual blocks with doubly residual stacking
- **Best for**: Interpretable forecasts with clear decomposition
- **Paper**: [NBEATS: Neural basis expansion analysis for interpretable time series forecasting](https://arxiv.org/abs/1905.10437)

### 2. **NHITS** (Neural Hierarchical Interpolation for Time Series)
- **Key Idea**: Multi-rate interpolation across different temporal scales
- **Fast**: More efficient than NBEATS (fewer parameters)
- **Architecture**: Hierarchical stacks with interpolation
- **Best for**: Long-horizon forecasting with speed requirements
- **Paper**: [NHITS: Neural Hierarchical Interpolation for Time Series Forecasting](https://arxiv.org/abs/2201.12886)

Let's start by training simple models with default parameters!

## 5. Train NBEATS with Default Parameters

We'll start with a simple NBEATS model using minimal configuration.

In [ ]:
# Train NBEATS with simple parameters
print("Training NBEATS...")

# Initialize model
# h = forecast horizon (14 days)
# input_size = number of historical steps to use (2 * h is a good default)
# max_steps = training iterations (keep low for speed)
models_nbeats = [
    NBEATS(
        h=test_size,
        input_size=2 * test_size,
        #max_steps=500,
        
        #early_stop_patience_steps=3
    )
]

# Create NeuralForecast object
nf_nbeats = NeuralForecast(models=models_nbeats, freq="D")

# Fit model
nf_nbeats.fit(df=df_train_nf)

# Generate forecasts
forecasts_nbeats = nf_nbeats.predict()

print("\n✅ NBEATS trained and forecasted!")
print(f"Forecast shape: {forecasts_nbeats.shape}")
forecasts_nbeats.head()

In [ ]:
# Visualize NBEATS forecasts for all stores
fig, axes = plt.subplots(5, 2, figsize=(15, 14))
axes = axes.flatten()

# Merge forecasts with actual test values
forecasts_nbeats_merged = forecasts_nbeats.reset_index().merge(
    df_test_nf[["unique_id", "ds", "y"]], on=["unique_id", "ds"]
)

mse_nbeats = {}

for idx, store in enumerate(stores):
    train_store = df_train_nf[df_train_nf["unique_id"] == store].sort_values("ds")
    test_store = df_test_nf[df_test_nf["unique_id"] == store].sort_values("ds")
    forecast_store = forecasts_nbeats_merged[
        forecasts_nbeats_merged["unique_id"] == store
    ].sort_values("ds")

    # Plot training data (last 90 days)
    train_context = train_store.tail(90)
    axes[idx].plot(
        train_context["ds"], train_context["y"], color="black", linewidth=1, alpha=0.7
    )

    # Plot actual test data
    axes[idx].plot(
        test_store["ds"],
        test_store["y"],
        color="black",
        linewidth=1.5,
        alpha=0.8,
        marker="o",
        markersize=2,
    )

    # Plot forecast
    axes[idx].plot(
        forecast_store["ds"],
        forecast_store["NBEATS"],
        color="#1f77b4",
        linewidth=2,
        linestyle="-",
        alpha=0.8,
    )

    # Mark split
    axes[idx].axvline(x=split_date, color="gray", linestyle="-", linewidth=1, alpha=0.5)

    # Calculate MSE
    mse = mean_squared_error(forecast_store["y"], forecast_store["NBEATS"])
    mse_nbeats[store] = mse

    axes[idx].set_title(f"{store} (MSE={mse:.0f})", fontsize=11, fontweight="bold")
    axes[idx].set_xlabel("Date", fontsize=8)
    axes[idx].set_ylabel("Sales", fontsize=8)
    axes[idx].tick_params(labelsize=7)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle(
    "NBEATS: All Stores", fontsize=14, fontweight="bold", y=1.00
)
plt.tight_layout()
plt.show()

avg_mse_nbeats = np.mean(list(mse_nbeats.values()))
print(f"\nAverage MSE across all stores: {avg_mse_nbeats:.2f}")

## 6. Train NHITS with Default Parameters

Now let's try NHITS, which is typically faster and more efficient than NBEATS.

In [ ]:
# Train NHITS with simple parameters
print("Training NHITS...")

models_nhits = [
    NHITS(
        h=test_size,
        input_size=2 * test_size,
        max_steps=500,
        early_stop_patience_steps=3
    )
]

# Create NeuralForecast object
nf_nhits = NeuralForecast(models=models_nhits, freq="D")

# Fit model
nf_nhits.fit(df=df_train_nf)

# Generate forecasts
forecasts_nhits = nf_nhits.predict()

print("\n✅ NHITS trained and forecasted!")
print(f"Forecast shape: {forecasts_nhits.shape}")
forecasts_nhits.head()

In [ ]:
# Visualize NHITS forecasts for all stores
fig, axes = plt.subplots(5, 2, figsize=(15, 14))
axes = axes.flatten()

# Merge forecasts with actual test values
forecasts_nhits_merged = forecasts_nhits.reset_index().merge(
    df_test_nf[["unique_id", "ds", "y"]], on=["unique_id", "ds"]
)

mse_nhits = {}

for idx, store in enumerate(stores):
    train_store = df_train_nf[df_train_nf["unique_id"] == store].sort_values("ds")
    test_store = df_test_nf[df_test_nf["unique_id"] == store].sort_values("ds")
    forecast_store = forecasts_nhits_merged[
        forecasts_nhits_merged["unique_id"] == store
    ].sort_values("ds")

    # Plot training data (last 90 days)
    train_context = train_store.tail(90)
    axes[idx].plot(
        train_context["ds"], train_context["y"], color="black", linewidth=1, alpha=0.7
    )

    # Plot actual test data
    axes[idx].plot(
        test_store["ds"],
        test_store["y"],
        color="black",
        linewidth=1.5,
        alpha=0.8,
        marker="o",
        markersize=2,
    )

    # Plot forecast
    axes[idx].plot(
        forecast_store["ds"],
        forecast_store["NHITS"],
        color="#ff7f0e",
        linewidth=2,
        linestyle="-",
        alpha=0.8,
    )

    # Mark split
    axes[idx].axvline(x=split_date, color="gray", linestyle="-", linewidth=1, alpha=0.5)

    # Calculate MSE
    mse = mean_squared_error(forecast_store["y"], forecast_store["NHITS"])
    mse_nhits[store] = mse

    axes[idx].set_title(f"{store} (MSE={mse:.0f})", fontsize=11, fontweight="bold")
    axes[idx].set_xlabel("Date", fontsize=8)
    axes[idx].set_ylabel("Sales", fontsize=8)
    axes[idx].tick_params(labelsize=7)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle(
    "NHITS: All Stores", fontsize=14, fontweight="bold", y=1.00
)
plt.tight_layout()
plt.show()

avg_mse_nhits = np.mean(list(mse_nhits.values()))
print(f"\nAverage MSE across all stores: {avg_mse_nhits:.2f}")

## 7. Compare Basic Models

Let's compare NBEATS and NHITS performance across all stores.

In [ ]:
# Print MSE comparison table
print("MSE Comparison: NBEATS vs NHITS (Default Parameters)")
print("=" * 60)
print(f"{'Store':<10} {'NBEATS':<20} {'NHITS':<20}")
print("-" * 60)
for store in stores:
    print(f"{store:<10} {mse_nbeats[store]:<20.2f} {mse_nhits[store]:<20.2f}")

# Average across stores
print("-" * 60)
print(f"{'Average':<10} {avg_mse_nbeats:<20.2f} {avg_mse_nhits:<20.2f}")

# Bar chart
fig, ax = plt.subplots(figsize=(10, 6))

model_names = ["NBEATS", "NHITS"]
avg_mses = [avg_mse_nbeats, avg_mse_nhits]
colors = ["#1f77b4", "#ff7f0e"]

bars = ax.bar(model_names, avg_mses, color=colors, alpha=0.8, edgecolor="black", linewidth=1.5)

# Add value labels
for bar, mse in zip(bars, avg_mses):
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{mse:.1f}",
        ha="center",
        va="bottom",
        fontsize=12,
        fontweight="bold",
    )

ax.set_ylabel("Average MSE", fontsize=12)
ax.set_title("Neural Network Models: Performance Comparison (Default Parameters)", fontsize=13, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")
plt.tight_layout()
plt.show()

print("\n💡 Both models capture patterns, but can we improve with automatic tuning?")

## 8. Automatic Hyperparameter Tuning with Auto Models

NeuralForecast provides **Auto- models** that automatically search for optimal hyperparameters!

### What are Auto Models?

Auto models use **Ray** or **Optuna** to automatically find the best hyperparameters:
- **AutoNBEATS**: Automatic hyperparameter optimization for NBEATS
- **AutoNHITS**: Automatic hyperparameter optimization for NHITS

### Key Parameters They Tune:
- **`input_size`**: How many historical steps to use (context window)
- **`learning_rate`**: Step size for gradient descent
- **`batch_size`**: Number of samples per training iteration
- **`max_steps`**: Number of training iterations
- **Architecture-specific**: stack types, hidden sizes, etc.

### How Auto Models Work:
1. You define a **search space** (or use defaults)
2. Specify **number of trials** (`num_samples`)
3. Auto model tries different configurations
4. Returns the **best model** based on validation loss

Let's use Auto models with a simple configuration!

## 9. AutoNBEATS: Automatic Hyperparameter Tuning

We'll use AutoNBEATS to automatically find the best hyperparameters.

**Key parameters**:
- `h`: Forecast horizon (14 days)
- `num_samples`: Number of hyperparameter configurations to try (keep low for speed)
- `config`: Custom search space (None = use defaults)

In [ ]:
# Train AutoNBEATS with automatic hyperparameter tuning
print("Training AutoNBEATS with automatic hyperparameter tuning...")
print("This will try multiple configurations and select the best one.\n")

# Initialize AutoNBEATS
# num_samples = number of hyperparameter configurations to try (3-5 for quick demo)
# config = search space (None uses sensible defaults)
models_auto_nbeats = [
    AutoNBEATS(
        h=test_size,
        num_samples=3,  # Try 3 different configurations (keep low for speed)
        config=None      # Use default search space
    )
]

# Create NeuralForecast object
nf_auto_nbeats = NeuralForecast(models=models_auto_nbeats, freq="D")

# Fit model (this will automatically tune hyperparameters)
nf_auto_nbeats.fit(df=df_train_nf)

# Generate forecasts
forecasts_auto_nbeats = nf_auto_nbeats.predict()

print("\n✅ AutoNBEATS trained and forecasted!")
print(f"Forecast shape: {forecasts_auto_nbeats.shape}")
forecasts_auto_nbeats.head()

In [ ]:
# Evaluate AutoNBEATS
forecasts_auto_nbeats_merged = forecasts_auto_nbeats.reset_index().merge(
    df_test_nf[["unique_id", "ds", "y"]], on=["unique_id", "ds"]
)

mse_auto_nbeats = {}

for store in stores:
    store_forecasts = forecasts_auto_nbeats_merged[
        forecasts_auto_nbeats_merged["unique_id"] == store
    ]
    mse = mean_squared_error(store_forecasts["y"], store_forecasts["AutoNBEATS"])
    mse_auto_nbeats[store] = mse

avg_mse_auto_nbeats = np.mean(list(mse_auto_nbeats.values()))

print("AutoNBEATS Results:")
print("=" * 60)
print(f"Average MSE: {avg_mse_auto_nbeats:.2f}")
print(f"\nComparison:")
print(f"  NBEATS (default):  {avg_mse_nbeats:.2f}")
print(f"  AutoNBEATS (tuned): {avg_mse_auto_nbeats:.2f}")

improvement = ((avg_mse_nbeats - avg_mse_auto_nbeats) / avg_mse_nbeats) * 100
print(f"  → Improvement: {improvement:.1f}%")

## 10. AutoNHITS: Automatic Hyperparameter Tuning

Now let's use AutoNHITS to automatically find the best hyperparameters for NHITS.

In [ ]:
# Train AutoNHITS with automatic hyperparameter tuning
print("Training AutoNHITS with automatic hyperparameter tuning...")
print("This will try multiple configurations and select the best one.\n")

# Initialize AutoNHITS
models_auto_nhits = [
    AutoNHITS(
        h=test_size,
        num_samples=3,  # Try 3 different configurations
        config=None      # Use default search space
    )
]

# Create NeuralForecast object
nf_auto_nhits = NeuralForecast(models=models_auto_nhits, freq="D")

# Fit model (this will automatically tune hyperparameters)
nf_auto_nhits.fit(df=df_train_nf)

# Generate forecasts
forecasts_auto_nhits = nf_auto_nhits.predict()

print("\n✅ AutoNHITS trained and forecasted!")
print(f"Forecast shape: {forecasts_auto_nhits.shape}")
forecasts_auto_nhits.head()

In [ ]:
# Evaluate AutoNHITS
forecasts_auto_nhits_merged = forecasts_auto_nhits.reset_index().merge(
    df_test_nf[["unique_id", "ds", "y"]], on=["unique_id", "ds"]
)

mse_auto_nhits = {}

for store in stores:
    store_forecasts = forecasts_auto_nhits_merged[
        forecasts_auto_nhits_merged["unique_id"] == store
    ]
    mse = mean_squared_error(store_forecasts["y"], store_forecasts["AutoNHITS"])
    mse_auto_nhits[store] = mse

avg_mse_auto_nhits = np.mean(list(mse_auto_nhits.values()))

print("AutoNHITS Results:")
print("=" * 60)
print(f"Average MSE: {avg_mse_auto_nhits:.2f}")
print(f"\nComparison:")
print(f"  NHITS (default):  {avg_mse_nhits:.2f}")
print(f"  AutoNHITS (tuned): {avg_mse_auto_nhits:.2f}")

improvement = ((avg_mse_nhits - avg_mse_auto_nhits) / avg_mse_nhits) * 100
print(f"  → Improvement: {improvement:.1f}%")

## 11. Compare All Models

Let's compare default models vs auto-tuned models.

In [ ]:
# Print comprehensive MSE comparison
print("MSE Comparison: Default vs Auto-Tuned Models")
print("=" * 80)
print(f"{'Store':<10} {'NBEATS':<15} {'AutoNBEATS':<15} {'NHITS':<15} {'AutoNHITS':<15}")
print("-" * 80)

for store in stores:
    print(
        f"{store:<10} {mse_nbeats[store]:<15.2f} {mse_auto_nbeats[store]:<15.2f} "
        f"{mse_nhits[store]:<15.2f} {mse_auto_nhits[store]:<15.2f}"
    )

# Average across stores
print("-" * 80)
print(
    f"{'Average':<10} {avg_mse_nbeats:<15.2f} {avg_mse_auto_nbeats:<15.2f} "
    f"{avg_mse_nhits:<15.2f} {avg_mse_auto_nhits:<15.2f}"
)

# Bar chart comparison
fig, ax = plt.subplots(figsize=(12, 6))

models = ["NBEATS\n(Default)", "AutoNBEATS\n(Tuned)", "NHITS\n(Default)", "AutoNHITS\n(Tuned)"]
avg_mses = [avg_mse_nbeats, avg_mse_auto_nbeats, avg_mse_nhits, avg_mse_auto_nhits]
colors = ["#8fc1e3", "#1f77b4", "#ffbb78", "#ff7f0e"]

bars = ax.bar(models, avg_mses, color=colors, alpha=0.8, edgecolor="black", linewidth=1.5)

# Add value labels
for bar, mse in zip(bars, avg_mses):
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{mse:.1f}",
        ha="center",
        va="bottom",
        fontsize=11,
        fontweight="bold",
    )

ax.set_ylabel("Average MSE", fontsize=12)
ax.set_title(
    "Impact of Automatic Hyperparameter Tuning on Neural Network Performance",
    fontsize=13,
    fontweight="bold",
)
ax.grid(True, alpha=0.3, axis="y")
plt.tight_layout()
plt.show()

# Calculate improvements
nbeats_improvement = ((avg_mse_nbeats - avg_mse_auto_nbeats) / avg_mse_nbeats) * 100
nhits_improvement = ((avg_mse_nhits - avg_mse_auto_nhits) / avg_mse_nhits) * 100

print(f"\n💡 Automatic Tuning Impact:")
print(f"  NBEATS: {nbeats_improvement:.1f}% improvement")
print(f"  NHITS: {nhits_improvement:.1f}% improvement")
print("  → Auto models automatically find better hyperparameters!")

## 12. Visualize Best Model

Let's visualize the best performing model for all stores.

In [ ]:
# Determine best model
all_avg_mses = {
    "NBEATS": avg_mse_nbeats,
    "AutoNBEATS": avg_mse_auto_nbeats,
    "NHITS": avg_mse_nhits,
    "AutoNHITS": avg_mse_auto_nhits
}

best_model_name = min(all_avg_mses, key=all_avg_mses.get)
best_avg_mse = all_avg_mses[best_model_name]

# Select best forecasts
if best_model_name == "NBEATS":
    best_forecasts = forecasts_nbeats_merged
    best_mse = mse_nbeats
elif best_model_name == "AutoNBEATS":
    best_forecasts = forecasts_auto_nbeats_merged
    best_mse = mse_auto_nbeats
elif best_model_name == "NHITS":
    best_forecasts = forecasts_nhits_merged
    best_mse = mse_nhits
else:
    best_forecasts = forecasts_auto_nhits_merged
    best_mse = mse_auto_nhits

print(f"Best Model: {best_model_name}")
print(f"Average MSE: {best_avg_mse:.2f}\n")

# Visualize best model for all stores
fig, axes = plt.subplots(5, 2, figsize=(15, 14))
axes = axes.flatten()

for idx, store in enumerate(stores):
    train_store = df_train_nf[df_train_nf["unique_id"] == store].sort_values("ds")
    test_store = df_test_nf[df_test_nf["unique_id"] == store].sort_values("ds")
    forecast_store = best_forecasts[
        best_forecasts["unique_id"] == store
    ].sort_values("ds")

    # Plot training data (last 90 days)
    train_context = train_store.tail(90)
    axes[idx].plot(
        train_context["ds"], train_context["y"], color="black", linewidth=1, alpha=0.7
    )

    # Plot actual test data
    axes[idx].plot(
        test_store["ds"],
        test_store["y"],
        color="black",
        linewidth=1.5,
        alpha=0.8,
        marker="o",
        markersize=2,
    )

    # Plot forecast
    color = "#2ca02c"
    axes[idx].plot(
        forecast_store["ds"],
        forecast_store[best_model_name],
        color=color,
        linewidth=2,
        linestyle="-",
        alpha=0.8,
    )

    # Mark split
    axes[idx].axvline(x=split_date, color="gray", linestyle="-", linewidth=1, alpha=0.5)

    mse = best_mse[store]
    axes[idx].set_title(f"{store} (MSE={mse:.0f})", fontsize=11, fontweight="bold")
    axes[idx].set_xlabel("Date", fontsize=8)
    axes[idx].set_ylabel("Sales", fontsize=8)
    axes[idx].tick_params(labelsize=7)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle(
    f"Best Model: {best_model_name}", fontsize=14, fontweight="bold", y=1.00
)
plt.tight_layout()
plt.show()

## 13. Summary and Key Takeaways

### What We Learned in Session 4:

#### 1. **NeuralForecast Library**
- User-friendly interface for neural forecasting
- Sklearn-like `.fit()` and `.predict()` methods
- Built on PyTorch with automatic GPU support

#### 2. **Neural Network Models**
- **NBEATS**: Interpretable, decomposes into trend/seasonality
- **NHITS**: Fast, efficient, hierarchical interpolation
- Both are **global models** - train once on all series

#### 3. **Automatic Hyperparameter Tuning**
- **AutoNBEATS** and **AutoNHITS** automatically find best parameters
- Use Ray/Optuna for efficient hyperparameter search
- Simply specify `num_samples` (number of trials)
- No manual tuning required!

#### 4. **When to Use Neural Networks**

**Use neural networks when:**
- ✅ You have **large datasets** (thousands of observations)
- ✅ Patterns are **complex and non-linear**
- ✅ You have **multiple related time series** (global modeling)
- ✅ You have **computational resources** (GPU recommended)

**Stick with simpler models when:**
- ⚠️ Dataset is **small** (<1000 observations)
- ⚠️ Patterns are **simple and linear**
- ⚠️ **Interpretability** is critical
- ⚠️ **Speed** is more important than accuracy

### Performance Summary:

| Model | Average MSE | Notes |
|-------|------------|-------|
| NBEATS (default) | Baseline | Fixed parameters |
| AutoNBEATS | Improved | Automatically tuned |
| NHITS (default) | Baseline | Fixed parameters |
| AutoNHITS | Improved | Automatically tuned |

### Key Principles:

1. **Start simple** - test with default parameters first
2. **Use Auto models** - let the library find optimal hyperparameters
3. **Global modeling** - neural networks train on all series together
4. **Validate properly** - ensure test period is representative
5. **Consider trade-offs** - accuracy vs speed vs interpretability

### Auto Models vs Manual Tuning:

**Advantages of Auto Models:**
- ✅ No expert knowledge required
- ✅ Automatic hyperparameter search
- ✅ Often find better configurations than manual tuning
- ✅ Save time and effort

**When to use manual tuning:**
- Domain-specific constraints (e.g., fixed input_size)
- Very limited computational budget
- Need precise control over all parameters

---

### 🎓 Congratulations!

You've completed Session 4 of the Time Series Forecasting Workshop. You now understand:
- How to use NeuralForecast for neural network models
- Popular architectures: NBEATS and NHITS
- **Automatic hyperparameter tuning** with AutoNBEATS and AutoNHITS
- When to use neural networks vs traditional ML

**Next Steps**: 
- Explore other models: LSTM, TFT, Transformers
- Try custom search spaces with Auto models
- Experiment with probabilistic forecasting
- Scale up with GPU acceleration! 🚀